In [1]:
from utils_1551 import Corpus, load_train_and_test, text2tokens

from bpemb import BPEmb
from uk_stopwords import STOP_WORDS
from typing import List, Dict
import numpy as np
import random

In [2]:
train, test = load_train_and_test()

In [3]:
dim = 300
bpemb_ua = BPEmb(lang="uk", dim=dim)

In [4]:
print(bpemb_ua.embed('щось там').shape)
print(bpemb_ua.encode('щось там'))

(3, 300)
['▁що', 'сь', '▁там']


In [5]:
s = "Ваша відповідь:  Повідомляємо, що відповідно до п. 5.7 Державного стандарту України 4123-2006 «Пристрій примусового зниження швидкості дорожньо-транспортної техніки на вулицях і дорогах. Загальні технічні вимоги» заборонено встановлювати пристрої примусового зниження швидкості руху транспортних засобів на магістральних вулицях, а просп. Григоренка має категорію магістральної вулиці районного значення."
print(text2tokens(s))

['Ваша', 'відповідь', 'Повідомляємо', 'що', 'відповідно', 'до', 'п', '5', '7', 'Державного', 'стандарту', 'України', '4123', '2006', 'Пристрій', 'примусового', 'зниження', 'швидкості', 'дорожньо', 'транспортної', 'техніки', 'на', 'вулицях', 'і', 'дорогах', 'Загальні', 'технічні', 'вимоги', 'заборонено', 'встановлювати', 'пристрої', 'примусового', 'зниження', 'швидкості', 'руху', 'транспортних', 'засобів', 'на', 'магістральних', 'вулицях', 'а', 'просп', 'Григоренка', 'має', 'категорію', 'магістральної', 'вулиці', 'районного', 'значення']


In [6]:
def text2vec(text: str):
    res = np.zeros(dim)
    words = text2tokens(text.lower())
    for w in words:
        if w in STOP_WORDS:
            continue
        for v in bpemb_ua.embed(w):
            res += v
    res /= max(np.linalg.norm(res), 1e-6)
    return res

print(text2vec(s).shape)

(300,)


In [7]:
def corpus2xy(corpus: Corpus)->(np.array, np.array):
    x, y = [], []
    for name, messages in corpus.items():
        for id_, text in messages:
            vec = text2vec(text)
            x.append(vec)
            y.append(name)
    return np.array(x), np.array(y)

def extract_labels(corpus: Corpus)->Dict[str, int]:
    res = {name:i for i, name in enumerate(corpus.keys())}
    return res

labels_dict = extract_labels(train)
def labels_to_ind(y: np.array)->np.array:
    return np.array([labels_dict[name] for name in y])

def shuffle_xy(x: np.array, y:np.array)->(np.array, np.array):
    assert len(x) == len(y)
    p = np.random.permutation(len(x))
    return x[p], y[p]

In [8]:
x_train, y_train = corpus2xy(train)

#print(x_train[:3])
print(y_train[:3])

['Інші-Подяки' 'Інші-Подяки' 'Інші-Подяки']


In [9]:
x_train, y_train = shuffle_xy(x_train, y_train)
#print(x_train[:3])
print(y_train[:3])

['Про-розгляд-звернень-громадян' 'Незадовільний-стан-опори-для-освітлення'
 'Обслуговування-та-технічний-стан-каналізаційних-колодязів']


In [10]:
y_train_idx = labels_to_ind(y_train)

In [11]:
print(x_train.shape, x_train.dtype)
print(y_train_idx.shape, y_train_idx.dtype)

(48120, 300) float64
(48120,) int32


In [12]:
x_test, y_test = corpus2xy(test)
y_test_idx = labels_to_ind(y_test)

In [13]:
#from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
#clf = LGBMClassifier(objective='multiclass', random_state=1974, min_child_samples=5)
#clf = LGBMClassifier(random_state=1974, num_leaves=512, min_child_samples=5)
clf = RandomForestClassifier(verbose=1, n_estimators=100, n_jobs=4)
clf.fit(x_train, y_train_idx)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.7min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [14]:
y_pred = clf.predict(x_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.6s finished


In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test_idx, y_pred))

C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.78      0.09      0.16        78
           1       0.34      0.36      0.35       182
           2       0.55      0.08      0.14        73
           3       1.00      0.32      0.48        19
           4       1.00      0.18      0.31        22
           5       1.00      0.16      0.28        25
           6       0.50      0.07      0.12        15
           7       0.46      0.78      0.58       161
           8       0.93      0.42      0.58        33
           9       0.83      0.21      0.33        24
          10       0.67      0.09      0.16        22
          11       0.62      0.22      0.32        82
          12       1.00      0.04      0.08        23
          13       1.00      0.38      0.55        34
          14       0.76      0.24      0.36        55
          15       0.50      0.12      0.20        16
          16       0.72      0.42      0.53       109
          17       0.63    